In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df_gowalla_edges = pd.read_csv('..\data\loc-gowalla_edges.txt\Gowalla_edges.txt', sep='\t', header=None)
df_gowalla_edges.columns = ['userid','friendship_links']
df_gowalla_edges.head()

,userid,friendship_links
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [4]:
df_gowalla = pd.read_csv('..\data\loc-gowalla_totalCheckins.txt\Gowalla_totalCheckins.txt', sep='\t', header=None)
df_gowalla.columns = ['userid','timestamp','latitude','longitude','spotid']

lon_min, lat_min, lon_max, lat_max = -74.2589, 40.4774, -73.7004, 40.9176
nyc_events = df_gowalla[(df_gowalla['longitude']>lon_min) & 
           (df_gowalla['longitude']<lon_max) & 
           (df_gowalla['latitude']>lat_min) & 
           (df_gowalla['latitude']<lat_max)]
venues = pd.read_csv('..\data\spots.txt\spots.txt', sep='\t', header=0)
nyc_events = pd.DataFrame.merge(nyc_events, venues[['spotid','spotname']], on='spotid', how="inner")
nyc_events.head()

,userid,timestamp,latitude,longitude,spotid,spotname
0,0,2010-10-12T00:21:28Z,40.643885,-73.782806,23261,JFK John F. Kennedy International
1,0,2010-10-07T15:27:40Z,40.643885,-73.782806,23261,JFK John F. Kennedy International
2,5,2010-08-16T03:47:20Z,40.643885,-73.782806,23261,JFK John F. Kennedy International
3,8,2010-10-06T20:23:14Z,40.643885,-73.782806,23261,JFK John F. Kennedy International
4,15,2010-10-13T04:10:36Z,40.643885,-73.782806,23261,JFK John F. Kennedy International


In [5]:
import operator
locFreq = {}

for friendId in df_gowalla_edges[df_gowalla_edges['userid']==5]['friendship_links']:
    friendLocs = nyc_events[nyc_events['userid']==friendId] 
    if not (friendLocs.empty): 
        for spotname in friendLocs['spotname']:
            if(locFreq.get(spotname) == None):
                locFreq[spotname] = 1
            else: 
                locFreq[spotname] = locFreq[spotname] + 1

# locFreq = sorted(locFreq.items(), key=operator.itemgetter(1), reverse=True)[:5] 

locFreqDf = pd.DataFrame(columns=['spotid', 'spotname', 'count'])

for key, value in locFreq.items():
    spotid=nyc_events[nyc_events['spotname']==key]['spotid'].iloc[0]
    locFreqDf = locFreqDf.append({'spotid': spotid , 'spotname': key, 'count': value}, ignore_index=True)

locFreqDf[:15]

,spotid,spotname,count
0,23261,JFK John F. Kennedy International,49
1,16907,Shake Shack,14
2,12973,Flatiron Building,5
3,341255,Emilio's Ballato Restaurant,1
4,260957,The Half Pint,4
5,1933724,New York University (NYU),4
6,105068,Market,2
7,34817,Westville,1
8,27836,The Chelsea Market,5
9,15079,The High Line,9
